# **Minesweeper Solver with z3**

The following notebook provides a method for solving Minesweeper using z3 as a Satisfiability Modulo Theories solver which I developed for my class on Artificial Intelligence at the Free University of Bolzano/Bozen.

Various board configurations can be queried here: https://davidnhill.github.io/JSMinesweeper.

In [1]:
from z3 import *

In [ ]:
## Environment ##
name: ailab-cop
channels:
  - conda-forge
  - nodefaults
dependencies:
  - python >3.8,<3.12
  - clingo
  - ipywidgets
  - jupyterlab
  - jupyterlab-git
  - jupytext
  - matplotlib
  - networkx
  - pandas
  - pip
  - pip:
      - minizinc
      - nographs
      - z3-solver

## **Minesweeper Puzzle Creator**

The following function encodes a specific starting board into the list-of-lists input required by the solver.

In [2]:
def get_grid(minefield: str) -> tuple[tuple]:
    return list(list(c for c in line) for line in map(str.strip, minefield.splitlines()) if len(line))

get_grid('''
    01?10001?
    01?100011
    011100000
    000000000
    111110011
    ????1001?
    ????3101?
    ?????211?
    ?????????
 ''')

[['0', '1', '?', '1', '0', '0', '0', '1', '?'],
 ['0', '1', '?', '1', '0', '0', '0', '1', '1'],
 ['0', '1', '1', '1', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['1', '1', '1', '1', '1', '0', '0', '1', '1'],
 ['?', '?', '?', '?', '1', '0', '0', '1', '?'],
 ['?', '?', '?', '?', '3', '1', '0', '1', '?'],
 ['?', '?', '?', '?', '?', '2', '1', '1', '?'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?']]

## **Minesweeper Solver**

The following function solves unambiguous starting instances of the Minesweeper game, here fixed at boardsizes no greater than 6x6.

In [3]:
def minesweepersolver(puzzle, total_mines):
    s = Solver()

    #Instantiating the Structure:
    structure = [[Bool(f"mine_{i+1}_{j+1}") for j in range(6)] for i in range(6)] #creates the matrix framework with variables of the Boolean type, given names relative to their coordinates 

    #Constraints #1 - Bombs and Values must Match 
    for i in range(6):
        for j in range(6):
            #Bomb Count:
            """For every square on the board, the function sums the total of 'True' squares (bomb locations), delimiting the search to only the board itself 
            by excluding hypothetical coordinate combinations which fall outside of the board. Takes the total and subtracts 1, if the square under review 
            has a bomb itself, so as to not add it in the bomb count of its neighbors."""
            count = sum(structure[x][y] for x in range(max(0, i-1), min(6, i+2)) for y in range(max(0, j-1), min(6, j+2))) - structure[i][j]

            #Constraint #1:
            """For every square on the board which is not unknown (and therefore has a value), if the Boolean value assigned to the square is 'True' (has a 
            bomb), then the constraint is that the square must be 'True'. Otherwise, the count of bombs in neighboring squares must equal the value assigned 
            to the square on the board. """
            if puzzle[i][j] != '?':
                s.add(If(structure[i][j], True, count == puzzle[i][j]))

    #Constraint #2 - Bomb Count cannot exceed Total
    """For every square on the board, if the Boolean value is 'True' (has a bomb), count +1, otherwise count +0. Takes the sum which must match the 
    total number of bombs."""
    s.add(sum([If(structure[i][j], 1, 0) for i in range(6) for j in range(6)]) == total_mines)

    #Check for Satisfation
    """If the solver creates a solution which satisfies the constraints, create a model with the Boolean values of the solution. The Boolean matrix then 
    gets updated with the solution values."""
    if s.check() == sat:
        model = s.model()
        solution = [[model.evaluate(structure[i][j]) for j in range(6)] for i in range(6)]

        """Transforms the solution into a 'X' (bomb) / '.' (no bomb) visual."""
        if solution:
            print('Solution:')
            for row in solution:
                print(' '.join('X' if cell == True else '.' for cell in row))
        
        return 
        
    else:
        print('No solution found.')

        return None

In [4]:
ms_puzzle = [['?', '?', '?', '?', '?', '?'],
 ['?', '1', '1', '2', '3', '?'],
 ['?', '1', '0', '0', '3', '?'],
 ['?', '1', '0', '0', '2', '?'],
 ['?', '2', '2', '3', '4', '?'],
 ['?', '?', '?', '?', '?', '?']]

minesweepersolver(ms_puzzle, 10)

Solution:
. . X . X .
. . . . . X
. . . . . X
X . . . . X
. . . . . .
. . X X X X


## **Minesweeper Solver for Ambiguity**

The following function solves ambiguous starting instances of the Minesweeper game. Ambiguity here implies that the starting instance of the game does not provide sufficient information to be able to solve the game outright. If the above solver is applied, it provides a potential solution, however it may differ from the actual solution of the board. To get around this, the player is iteratively prompted to click on cells known to be safe, so as to uncover additional information which will aid in arriving at the specific solution for the board. Because the boardsizes which have implicit ambiguity tend to be larger, this function also has the user specify the dimensions of the board.

In [1]:
def minesweepersolver_amb(puzzle, total_mines, boardsize_x, boardsize_y):
    count = sum(1 for row in puzzle for item in row if item == '?')

    while count > total_mines:
        s = Solver()

        #Instantiating the Structure:
        structure = [[Bool(f"mine_{i+1}_{j+1}") for j in range(boardsize_x)] for i in range(boardsize_y)] #creates the matrix framework with variables of the Boolean type, given names relative to their coordinates 

        #Constraints #1 - Bombs and Values must Match 
        for i in range(boardsize_y):
            for j in range(boardsize_x):
                #Bomb Count:
                """For every square on the board, the function sums the total of 'True' squares (bomb locations), delimiting the search to only the board itself 
                by excluding hypothetical coordinate combinations which fall outside of the board. Takes the total and subtracts 1, if the square under review 
                has a bomb itself, so as to not add it in the bomb count of its neighbors."""
                count = sum(structure[x][y] for x in range(max(0, i-1), min(boardsize_y, i+2)) for y in range(max(0, j-1), min(boardsize_x, j+2))) - structure[i][j]

                #Constraint #1:
                """For every square on the board which is not unknown (and therefore has a value), if the Boolean value assigned to the square is 'True' (has a 
                bomb), then the constraint is that the square must be 'True'. Otherwise, the count of bombs in neighboring squares must equal the value assigned 
                to the square on the board. """
                if puzzle[i][j] != '?':
                    s.add(If(structure[i][j], True, count == puzzle[i][j]))

        #Constraint #2 - Bomb Count cannot exceed Total
        """For every square on the board, if the Boolean value is 'True' (has a bomb), count +1, otherwise count +0. Takes the sum which must match the 
        total number of bombs."""
        s.add(sum([If(structure[i][j], 1, 0) for i in range(boardsize_y) for j in range(boardsize_x)]) == total_mines)


        #Iteration through Negation to Find True Solution
        """While the total number of unknown cells is greater than the total number of mines, iterate through each square on the board asserting that there is a bomb. If this
        assertion cannot satisfy the constraints, there is undoubtedly no bomb in the square. The player is then prompted to click on the cell and insert the new number. The loop
        continues until the total number of unkown cells equals the total number of bombs, implying that the solution has been reached. There are two iteration patterns: (1) without 
        the "stayin" Boolean variable, which iterates based on rounds, i.e. the safe squares are calculated one time each round and asked subsequently in order; and (2) with the "stayin"
        Boolean variable, which iterates every input that the player gives and therefore the safe squares are prompted linearly."""
        # stayin = True
        for i in range(boardsize_y):
            for j in range(boardsize_x):
                if puzzle[i][j] == '?':
                    s.push()
                    s.add(structure[i][j] == True)
                    if s.check() == unsat:
                        pippinput = str(input(f'Click on the puzzle at {(i,j)} and enter the number:'))
                        puzzle[i][j] = pippinput
                        # stayin = False
                        count = sum(1 for row in puzzle for item in row if item == '?')                        
                    s.pop()
            #     if not stayin:
            #         break
            # if not stayin:
            #     break


    #Check for Satisfation
    """If the solver creates a solution which satisfies the constraints, create a model with the Boolean values of the solution. The Boolean matrix then 
    gets updated with the solution values."""
    if s.check() == sat:
        model = s.model()
        solution = [[model.evaluate(structure[i][j]) for j in range(boardsize_x)] for i in range(boardsize_y)]

        """Transforms the solution into a 'X' (bomb) / '.' (no bomb) visual."""
        if solution:
            print('Solution:')
            for row in solution:
                print(' '.join('X' if cell == True else '.' for cell in row))
        
        return 
    
    else:
        print('No solution found.')

        return 

In [12]:
ms_puzzle = [['0', '1', '?', '1', '0', '0', '0', '1', '?'],
 ['0', '1', '?', '1', '0', '0', '0', '1', '1'],
 ['0', '1', '1', '1', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['1', '1', '1', '1', '1', '0', '0', '1', '1'],
 ['?', '?', '?', '?', '1', '0', '0', '1', '?'],
 ['?', '?', '?', '?', '3', '1', '0', '1', '?'],
 ['?', '?', '?', '?', '?', '2', '1', '1', '?'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?']]

minesweepersolver_amb(ms_puzzle, total_mines = 10, boardsize_x = 9, boardsize_y = 9)

Solution:
. . . . . . . . X
. . X . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
X . . X . . . . X
. . . . . . . . .
X . . X X . . . .
. . X . . . X . .
